<a href="https://colab.research.google.com/github/mevivekmevivek/Drug-Efficacy-Prediction/blob/main/rna_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
da

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving rna.pickle to rna (4).pickle


In [ ]:
import pickle
pickle_in = open("rna.pickle","rb")
rna_t = pickle.load(pickle_in)

In [ ]:

rna_t

symbol,CELL_LINE_NAME,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2ML1-AS2,A2MP1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AACSP1,AADAC,AADACL2,AADACL2-AS1,AADACL3,AADACL4,AADACP1,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AARSP1,AASDH,AASDHPPT,AASS,AATBC,AATF,...,ZSCAN16,ZSCAN16-AS1,ZSCAN18,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM5P1,ZSWIM5P2,ZSWIM5P3,ZSWIM6,ZSWIM7,ZSWIM8,ZSWIM8-AS1,ZSWIM9,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZYXP1,ZZEF1,ZZZ3
MEC-1,MEC-1,0,0,0,0,0,0,0,0,0,0,0,0,7,4,0,0,0,0,0,0,0,0,5,20,1,2,25,0,11,0,203,10,4,0,4,10,0,0,23,...,7,3,0,0,4,1,0,5,3,7,4,0,1,4,0,2,3,0,0,0,0,2,1,7,0,5,4,10,11,72,0,2,2,0,0,2,8,0,3,4
NBsusSR,NBsusSR,0,1,0,0,0,0,0,0,0,0,5,0,25,2,0,0,0,0,0,6,0,2,0,9,1,4,31,0,22,0,53,8,2,0,3,15,5,0,14,...,3,2,4,3,6,0,1,4,7,5,2,0,5,5,0,2,7,2,0,0,0,21,3,10,0,4,8,24,9,30,1,3,15,0,0,8,15,0,6,12
M14,M14,0,3,0,8,0,0,0,0,0,0,0,0,18,3,0,0,0,0,0,0,0,4,8,21,2,2,35,0,16,0,20,10,1,0,2,11,7,0,24,...,1,2,0,1,5,1,0,3,4,3,1,0,2,4,0,1,1,0,0,0,0,3,5,6,0,2,2,21,23,79,0,2,4,3,0,8,51,0,7,12
MDA-MB-134-VI,MDA-MB-134-VI,0,2,0,0,1,0,0,2,0,0,0,0,22,8,0,0,0,0,0,0,0,8,2,31,1,101,34,0,17,0,19,6,4,0,4,5,0,0,7,...,12,3,8,1,4,2,0,3,12,7,6,2,1,4,0,1,2,1,0,0,0,5,5,25,0,1,7,4,10,72,1,2,7,1,0,3,6,0,3,7
MCC26,MCC26,0,2,0,23,0,0,0,0,0,0,3,0,7,1,0,0,0,0,0,0,0,3,5,16,4,3,32,0,18,1,67,7,1,0,2,8,13,0,11,...,2,1,0,1,5,0,0,4,3,4,2,3,4,5,0,1,3,2,0,0,0,8,3,7,0,2,5,15,11,29,0,2,4,4,0,7,141,0,3,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H9,H9,0,0,0,0,0,0,0,0,0,0,0,0,9,5,0,0,0,0,0,0,0,0,7,34,3,2,37,0,19,0,90,9,4,0,3,7,0,0,34,...,3,1,0,0,7,2,1,11,3,7,3,0,5,7,0,5,4,0,0,0,0,1,9,6,1,29,11,7,9,57,3,7,10,5,0,4,44,0,18,6
P3HR-1,P3HR-1,0,0,0,0,0,0,0,0,0,0,1,0,30,1,0,0,0,0,0,0,0,0,0,22,0,1,39,0,11,0,60,6,5,0,3,13,0,0,34,...,5,1,0,1,2,1,0,3,2,7,1,0,1,5,0,2,1,0,0,0,0,1,5,6,0,3,9,12,23,61,0,0,3,0,0,2,1,0,9,6
HUH-6-clone5,HUH-6-clone5,0,0,5,13,0,0,0,0,0,0,0,0,23,4,0,0,0,0,0,0,1,5,7,10,2,3,33,0,17,0,70,5,1,0,4,7,3,0,12,...,4,1,8,0,6,1,0,3,3,2,1,0,2,3,0,1,1,2,0,0,0,5,3,5,0,3,4,11,11,29,0,2,8,2,0,4,19,0,2,8
GR-ST,GR-ST,0,4,0,0,0,0,0,0,0,0,0,0,19,8,0,0,0,0,0,0,0,0,0,36,3,2,24,0,14,0,167,4,2,0,5,9,0,0,26,...,7,3,0,1,7,3,0,7,3,18,5,1,8,3,0,2,6,4,0,0,0,0,5,9,0,10,9,23,26,50,1,6,14,0,0,7,8,0,20,15


In [ ]:
rna_t.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1047 entries, MEC-1 to MM1S
Columns: 37280 entries, CELL_LINE_NAME to ZZZ3
dtypes: object(1), uint8(37279)
memory usage: 37.2+ MB


In [ ]:
print (~rna_t.columns.duplicated())
rna_t = rna_t.loc[:, ~rna_t.columns.duplicated()]


[ True  True  True ...  True  True  True]


In [ ]:
rna_t.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1047 entries, MEC-1 to MM1S
Columns: 37278 entries, CELL_LINE_NAME to ZZZ3
dtypes: object(1), uint8(37277)
memory usage: 37.2+ MB


In [ ]:
rna_t

symbol,CELL_LINE_NAME,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2ML1-AS2,A2MP1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AACSP1,AADAC,AADACL2,AADACL2-AS1,AADACL3,AADACL4,AADACP1,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AARSP1,AASDH,AASDHPPT,AASS,AATBC,AATF,...,ZSCAN16,ZSCAN16-AS1,ZSCAN18,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM5P1,ZSWIM5P2,ZSWIM5P3,ZSWIM6,ZSWIM7,ZSWIM8,ZSWIM8-AS1,ZSWIM9,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZYXP1,ZZEF1,ZZZ3
MEC-1,MEC-1,0,0,0,0,0,0,0,0,0,0,0,0,7,4,0,0,0,0,0,0,0,0,5,20,1,2,25,0,11,0,203,10,4,0,4,10,0,0,23,...,7,3,0,0,4,1,0,5,3,7,4,0,1,4,0,2,3,0,0,0,0,2,1,7,0,5,4,10,11,72,0,2,2,0,0,2,8,0,3,4
NBsusSR,NBsusSR,0,1,0,0,0,0,0,0,0,0,5,0,25,2,0,0,0,0,0,6,0,2,0,9,1,4,31,0,22,0,53,8,2,0,3,15,5,0,14,...,3,2,4,3,6,0,1,4,7,5,2,0,5,5,0,2,7,2,0,0,0,21,3,10,0,4,8,24,9,30,1,3,15,0,0,8,15,0,6,12
M14,M14,0,3,0,8,0,0,0,0,0,0,0,0,18,3,0,0,0,0,0,0,0,4,8,21,2,2,35,0,16,0,20,10,1,0,2,11,7,0,24,...,1,2,0,1,5,1,0,3,4,3,1,0,2,4,0,1,1,0,0,0,0,3,5,6,0,2,2,21,23,79,0,2,4,3,0,8,51,0,7,12
MDA-MB-134-VI,MDA-MB-134-VI,0,2,0,0,1,0,0,2,0,0,0,0,22,8,0,0,0,0,0,0,0,8,2,31,1,101,34,0,17,0,19,6,4,0,4,5,0,0,7,...,12,3,8,1,4,2,0,3,12,7,6,2,1,4,0,1,2,1,0,0,0,5,5,25,0,1,7,4,10,72,1,2,7,1,0,3,6,0,3,7
MCC26,MCC26,0,2,0,23,0,0,0,0,0,0,3,0,7,1,0,0,0,0,0,0,0,3,5,16,4,3,32,0,18,1,67,7,1,0,2,8,13,0,11,...,2,1,0,1,5,0,0,4,3,4,2,3,4,5,0,1,3,2,0,0,0,8,3,7,0,2,5,15,11,29,0,2,4,4,0,7,141,0,3,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H9,H9,0,0,0,0,0,0,0,0,0,0,0,0,9,5,0,0,0,0,0,0,0,0,7,34,3,2,37,0,19,0,90,9,4,0,3,7,0,0,34,...,3,1,0,0,7,2,1,11,3,7,3,0,5,7,0,5,4,0,0,0,0,1,9,6,1,29,11,7,9,57,3,7,10,5,0,4,44,0,18,6
P3HR-1,P3HR-1,0,0,0,0,0,0,0,0,0,0,1,0,30,1,0,0,0,0,0,0,0,0,0,22,0,1,39,0,11,0,60,6,5,0,3,13,0,0,34,...,5,1,0,1,2,1,0,3,2,7,1,0,1,5,0,2,1,0,0,0,0,1,5,6,0,3,9,12,23,61,0,0,3,0,0,2,1,0,9,6
HUH-6-clone5,HUH-6-clone5,0,0,5,13,0,0,0,0,0,0,0,0,23,4,0,0,0,0,0,0,1,5,7,10,2,3,33,0,17,0,70,5,1,0,4,7,3,0,12,...,4,1,8,0,6,1,0,3,3,2,1,0,2,3,0,1,1,2,0,0,0,5,3,5,0,3,4,11,11,29,0,2,8,2,0,4,19,0,2,8
GR-ST,GR-ST,0,4,0,0,0,0,0,0,0,0,0,0,19,8,0,0,0,0,0,0,0,0,0,36,3,2,24,0,14,0,167,4,2,0,5,9,0,0,26,...,7,3,0,1,7,3,0,7,3,18,5,1,8,3,0,2,6,4,0,0,0,0,5,9,0,10,9,23,26,50,1,6,14,0,0,7,8,0,20,15


In [ ]:

# checking for columns with same value across 223 drugs
rna_same_value =[]
rna_new_features=[]
for x in rna_t.columns:
    if rna_t[x].nunique() ==1:
        rna_same_value.append(x)
    else :
        rna_new_features.append(x)

In [ ]:
print(len(rna_same_value))
print(len(rna_new_features))

8618
28660


In [ ]:
print(rna_same_value)

['AARSP1', 'ABCD1P2', 'ABCD1P3', 'ABCD1P5', 'ABCF2P1', 'ABCF2P2', 'ABHD16B', 'ABHD17AP1', 'ABHD17AP7', 'ABHD17AP9', 'ABI1P1', 'ABT1P1', 'ACE3P', 'ACOD1', 'ACSM5P1', 'ACTBP6', 'ACTBP9', 'ACTBP14', 'ACTG1P2', 'ACTG1P4', 'ACTG1P9', 'ACTG1P11', 'ACTG1P12', 'ACTG1P16', 'ACTG1P18', 'ACTG1P19', 'ACTG1P21', 'ACTL7A', 'ACTP1', 'ACTR3BP1', 'ACTR3BP2', 'ACTR3BP3', 'ACTR3BP4', 'ACTR3BP5', 'ACTR3BP6', 'ACTR3P2', 'ACTR3P3', 'ACTR6P1', 'ACTRT1', 'ADAD1P2', 'ADAM1A', 'ADAM3A', 'ADAM3B', 'ADAM5', 'ADAM6', 'ADAM20', 'ADAM20P2', 'ADAM21P1', 'ADAM24P', 'ADAM30', 'ADGRF5P2', 'ADGRL3-AS1', 'ADH5P2', 'ADH5P3', 'ADH5P4', 'ADI1P1', 'ADI1P2', 'ADI1P3', 'ADIPOQ-AS1', 'ADIPOR1P1', 'ADIPOR1P2', 'AGBL1-AS1', 'AGGF1P4', 'AGGF1P5', 'AGGF1P7', 'AGGF1P9', 'AGKP1', 'AGPAT5P1', 'AHCYP2', 'AHCYP3', 'AHCYP4', 'AHCYP5', 'AHCYP7', 'AHCYP8', 'AIG1P1', 'AIMP1P1', 'AIRN', 'AK2P2', 'AK3P4', 'AK4P4', 'AK4P5', 'AK4P6', 'AK6P1', 'AK6P2', 'AKIRIN1P1', 'AKIRIN1P2', 'AKIRIN2P1', 'AKR1B1P2', 'AKR1B1P6', 'AKR1B1P8', 'AKR1B10P1', 'AKR1B1

In [ ]:
rna_t = rna_t[rna_new_features]

In [ ]:
rna_t

symbol,CELL_LINE_NAME,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2ML1-AS2,A2MP1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AACSP1,AADAC,AADACL2,AADACL2-AS1,AADACL3,AADACL4,AADACP1,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATBC,AATF,AATK,...,ZSCAN10,ZSCAN12,ZSCAN12P1,ZSCAN16,ZSCAN16-AS1,ZSCAN18,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM5P3,ZSWIM6,ZSWIM7,ZSWIM8,ZSWIM8-AS1,ZSWIM9,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZZEF1,ZZZ3
MEC-1,MEC-1,0,0,0,0,0,0,0,0,0,0,0,0,7,4,0,0,0,0,0,0,0,0,5,20,1,2,25,0,11,0,203,10,4,4,10,0,0,23,0,...,0,3,0,7,3,0,0,4,1,0,5,3,7,4,0,1,4,0,2,3,0,0,2,1,7,0,5,4,10,11,72,0,2,2,0,0,2,8,3,4
NBsusSR,NBsusSR,0,1,0,0,0,0,0,0,0,0,5,0,25,2,0,0,0,0,0,6,0,2,0,9,1,4,31,0,22,0,53,8,2,3,15,5,0,14,6,...,0,1,0,3,2,4,3,6,0,1,4,7,5,2,0,5,5,0,2,7,2,0,21,3,10,0,4,8,24,9,30,1,3,15,0,0,8,15,6,12
M14,M14,0,3,0,8,0,0,0,0,0,0,0,0,18,3,0,0,0,0,0,0,0,4,8,21,2,2,35,0,16,0,20,10,1,2,11,7,0,24,0,...,0,1,0,1,2,0,1,5,1,0,3,4,3,1,0,2,4,0,1,1,0,0,3,5,6,0,2,2,21,23,79,0,2,4,3,0,8,51,7,12
MDA-MB-134-VI,MDA-MB-134-VI,0,2,0,0,1,0,0,2,0,0,0,0,22,8,0,0,0,0,0,0,0,8,2,31,1,101,34,0,17,0,19,6,4,4,5,0,0,7,1,...,0,1,5,12,3,8,1,4,2,0,3,12,7,6,2,1,4,0,1,2,1,0,5,5,25,0,1,7,4,10,72,1,2,7,1,0,3,6,3,7
MCC26,MCC26,0,2,0,23,0,0,0,0,0,0,3,0,7,1,0,0,0,0,0,0,0,3,5,16,4,3,32,0,18,1,67,7,1,2,8,13,0,11,0,...,0,2,0,2,1,0,1,5,0,0,4,3,4,2,3,4,5,0,1,3,2,0,8,3,7,0,2,5,15,11,29,0,2,4,4,0,7,141,3,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H9,H9,0,0,0,0,0,0,0,0,0,0,0,0,9,5,0,0,0,0,0,0,0,0,7,34,3,2,37,0,19,0,90,9,4,3,7,0,0,34,0,...,0,1,0,3,1,0,0,7,2,1,11,3,7,3,0,5,7,0,5,4,0,0,1,9,6,1,29,11,7,9,57,3,7,10,5,0,4,44,18,6
P3HR-1,P3HR-1,0,0,0,0,0,0,0,0,0,0,1,0,30,1,0,0,0,0,0,0,0,0,0,22,0,1,39,0,11,0,60,6,5,3,13,0,0,34,0,...,0,0,1,5,1,0,1,2,1,0,3,2,7,1,0,1,5,0,2,1,0,0,1,5,6,0,3,9,12,23,61,0,0,3,0,0,2,1,9,6
HUH-6-clone5,HUH-6-clone5,0,0,5,13,0,0,0,0,0,0,0,0,23,4,0,0,0,0,0,0,1,5,7,10,2,3,33,0,17,0,70,5,1,4,7,3,0,12,4,...,0,1,0,4,1,8,0,6,1,0,3,3,2,1,0,2,3,0,1,1,2,0,5,3,5,0,3,4,11,11,29,0,2,8,2,0,4,19,2,8
GR-ST,GR-ST,0,4,0,0,0,0,0,0,0,0,0,0,19,8,0,0,0,0,0,0,0,0,0,36,3,2,24,0,14,0,167,4,2,5,9,0,0,26,0,...,0,6,1,7,3,0,1,7,3,0,7,3,18,5,1,8,3,0,2,6,4,0,0,5,9,0,10,9,23,26,50,1,6,14,0,0,7,8,20,15


In [ ]:
rna_t.isnull().values.any()

False

In [ ]:
rna_y = rna_t['CELL_LINE_NAME']
rna_t.drop(columns =['CELL_LINE_NAME'], inplace = True)
rna_y

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


MEC-1                    MEC-1
NBsusSR                NBsusSR
M14                        M14
MDA-MB-134-VI    MDA-MB-134-VI
MCC26                    MCC26
                     ...      
H9                          H9
P3HR-1                  P3HR-1
HUH-6-clone5      HUH-6-clone5
GR-ST                    GR-ST
MM1S                      MM1S
Name: CELL_LINE_NAME, Length: 1047, dtype: object

In [ ]:
rna_scaled = minmax_scale(rna_t, axis = 0)

In [ ]:
X_train_rna, X_test_rna, Y_train_rna, Y_test_rna = train_test_split(rna_scaled, rna_y, train_size = 0.9, random_state = seed(2017))

In [ ]:
X_train_rna

array([[0.        , 0.        , 0.        , ..., 0.31325301, 0.1       ,
        0.09090909],
       [0.        , 0.07692308, 0.        , ..., 0.20481928, 0.3       ,
        0.27272727],
       [0.        , 0.        , 0.        , ..., 0.12048193, 0.15      ,
        0.06060606],
       ...,
       [0.        , 0.        , 0.        , ..., 0.03614458, 0.25      ,
        0.09090909],
       [0.        , 0.        , 0.        , ..., 0.09236948, 0.15      ,
        0.12121212],
       [0.        , 0.        , 0.        , ..., 0.30923695, 0.3       ,
        0.18181818]])

In [ ]:
mcol = X_train_rna.shape[1]
encoding_dim1 = 100

In [ ]:
from tensorflow.keras.layers import LeakyReLU
input_dim1 = Input(shape = (mcol, ))

# Encoder Layers
#encoded1 = Dense(20000)(input_dim1)
#encoded1= LeakyReLU(alpha=0.3)(encoded1)
#encoded2 = Dense(15000, activation = 'relu')(input_dim1)
#encoded3 = Dense(10000, activation = 'relu')(encoded2)
#encoded4 = Dense(5000)(encoded1)
#encoded4= LeakyReLU(alpha=0.3)(encoded4)
#encoded5 = Dense(2000, activation = 'relu')(encoded2)
#encoded6 = Dense(1750, activation = 'relu')(encoded5)
#encoded7 = Dense(1500, activation = 'relu')(encoded6)
#encoded8 = Dense(1250, activation = 'relu')(encoded7)
#encoded9 = Dense(1000)(encoded1)
#encoded9= LeakyReLU(alpha=0.3)(encoded9)
#encoded10 = Dense(750, activation = 'relu')(encoded9)
#encoded11 = Dense(500, activation = 'relu')(encoded10)
#encoded12 = Dense(250, activation = 'relu')(encoded9)
encoded13 = Dense(encoding_dim1)(input_dim1)
encoded13= LeakyReLU(alpha=0.3)(encoded13)

encoder1 = Model(inputs = input_dim1, outputs = encoded13)

# Decoder Layers
#decoded1 = Dense(mcol, activation = 'relu')(encoded13)
#decoded2 = Dense(500, activation = 'relu')(decoded1)
#decoded3 = Dense(750, activation = 'relu')(decoded2)
#decoded4 = Dense(1000, activation = 'relu')(encoded13)
#decoded5 = Dense(1250, activation = 'relu')(decoded4)
#decoded6 = Dense(1500, activation = 'relu')(decoded5)
#decoded7 = Dense(1750, activation = 'relu')(decoded6)
#decoded8 = Dense(2000, activation = 'relu')(encoded13)
#decoded9 = Dense(5000, activation = 'relu')(decoded4)
#decoded10 = Dense(10000, activation = 'relu')(decoded9)
#decoded11 = Dense(15000, activation = 'relu')(decoded1)
#decoded12 = Dense(20000, activation = 'relu')(encoded13)
decoded13 = Dense(mcol, activation = 'relu')(encoded13)

# Combine Encoder and Deocder layers
autoencoder1 = Model(inputs = input_dim1, outputs = decoded13)

# Compile the Model
autoencoder1.compile(optimizer = 'adam', loss = 'mae')

In [ ]:
autoencoder1.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 28659)]           0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               2866000   
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 28659)             2894559   
Total params: 5,760,559
Trainable params: 5,760,559
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# epo = [15,30,50]
# for l in epo:
autoencoder1.fit(X_train_rna, X_train_rna, epochs = 30, batch_size = 32, shuffle = False, validation_data = (X_test_rna, X_test_rna))

Epoch 1/30
30/30 [==============================] - 3s 77ms/step - loss: 0.0504 - val_loss: 0.0407
Epoch 2/30
30/30 [==============================] - 2s 72ms/step - loss: 0.0401 - val_loss: 0.0364
Epoch 3/30
30/30 [==============================] - 2s 74ms/step - loss: 0.0371 - val_loss: 0.0355
Epoch 4/30
30/30 [==============================] - 2s 70ms/step - loss: 0.0351 - val_loss: 0.0347
Epoch 5/30
30/30 [==============================] - 2s 72ms/step - loss: 0.0341 - val_loss: 0.0347
Epoch 6/30
30/30 [==============================] - 2s 72ms/step - loss: 0.0336 - val_loss: 0.0334
Epoch 7/30
30/30 [==============================] - 2s 72ms/step - loss: 0.0335 - val_loss: 0.0375
Epoch 8/30
30/30 [==============================] - 2s 72ms/step - loss: 0.0350 - val_loss: 0.0367
Epoch 9/30
30/30 [==============================] - 2s 71ms/step - loss: 0.0343 - val_loss: 0.0335
Epoch 10/30
30/30 [==============================] - 2s 72ms/step - loss: 0.0334 - val_loss: 0.0332
Epoch 11/

In [ ]:

encoded_input1 = Input(shape = (encoding_dim1, ))

In [ ]:
encoded_train1 = pd.DataFrame(encoder1.predict(rna_scaled))
encoded_train1 = encoded_train1.add_prefix('rna_feature_')

In [ ]:
encoded_train_columns = encoded_train1.columns
for l in encoded_train_columns:
  if (encoded_train1[l] == 0).all():
    print("TRUE", l)

In [ ]:
encoded_train1

,rna_feature_0,rna_feature_1,rna_feature_2,rna_feature_3,rna_feature_4,rna_feature_5,rna_feature_6,rna_feature_7,rna_feature_8,rna_feature_9,rna_feature_10,rna_feature_11,rna_feature_12,rna_feature_13,rna_feature_14,rna_feature_15,rna_feature_16,rna_feature_17,rna_feature_18,rna_feature_19,rna_feature_20,rna_feature_21,rna_feature_22,rna_feature_23,rna_feature_24,rna_feature_25,rna_feature_26,rna_feature_27,rna_feature_28,rna_feature_29,rna_feature_30,rna_feature_31,rna_feature_32,rna_feature_33,rna_feature_34,rna_feature_35,rna_feature_36,rna_feature_37,rna_feature_38,rna_feature_39,...,rna_feature_60,rna_feature_61,rna_feature_62,rna_feature_63,rna_feature_64,rna_feature_65,rna_feature_66,rna_feature_67,rna_feature_68,rna_feature_69,rna_feature_70,rna_feature_71,rna_feature_72,rna_feature_73,rna_feature_74,rna_feature_75,rna_feature_76,rna_feature_77,rna_feature_78,rna_feature_79,rna_feature_80,rna_feature_81,rna_feature_82,rna_feature_83,rna_feature_84,rna_feature_85,rna_feature_86,rna_feature_87,rna_feature_88,rna_feature_89,rna_feature_90,rna_feature_91,rna_feature_92,rna_feature_93,rna_feature_94,rna_feature_95,rna_feature_96,rna_feature_97,rna_feature_98,rna_feature_99
0,-0.113710,0.015776,0.255900,-0.353810,-0.684555,-0.505022,-0.934096,-0.262980,0.328180,-0.888463,-0.213263,-0.112308,-0.193263,-0.697479,-0.204158,-0.475362,-0.459413,-0.713710,-0.043445,-0.604697,-0.441277,-0.326982,-0.331256,-0.468288,-0.481006,2.817312,3.122547,0.326295,-0.087718,-0.157410,-0.170199,-0.666725,-0.786257,-0.207826,-0.336046,-0.392179,-0.224414,0.245889,-0.173383,-0.487124,...,-1.002041,-0.023796,3.221149,-0.376878,-0.411979,-0.279865,-0.261245,-0.345919,-1.033504,-0.168441,-2.014476,-0.452390,1.168029,-0.148469,-1.168720,-0.884369,-0.106900,1.025782,-0.015760,-1.974355,-0.704214,-0.548893,-0.427587,-0.700450,-0.257901,-0.387663,-0.345593,-1.205233,-0.079529,-0.405452,0.008337,-0.692550,-0.049275,-0.354898,-0.261631,0.314960,-0.014273,-0.237338,-0.313053,-0.426225
1,-0.025810,-0.012047,-0.304307,0.070642,-0.540065,-0.388465,-1.128392,-0.104629,-0.889848,-0.851125,-0.287239,0.048427,-0.069018,-1.194497,-0.095489,-0.241955,-0.437581,-0.493676,0.198132,-0.078260,-0.344380,-1.193587,-0.721444,-0.492744,0.355221,-0.191624,-0.895223,0.531082,4.812373,-0.961698,-0.400196,-0.368726,-0.836760,-0.073466,-0.350438,0.095026,-0.513887,-0.888748,1.276347,-0.465436,...,-0.278521,-0.005615,-0.021343,-0.469225,-0.094063,0.020835,-0.649219,-0.573378,-1.085598,-0.440069,-1.363173,-0.543473,1.204380,-0.234619,-0.419783,-0.287695,0.392210,-0.205331,2.549847,0.056657,0.104438,-1.500150,-0.361864,0.433886,1.374583,-0.607971,-0.454727,-1.956478,-0.516387,0.583284,-0.262945,1.225905,0.551230,-0.066023,-0.161026,-0.626396,-0.135961,1.877104,-0.575734,-0.646045
2,-0.106869,0.272746,0.061858,-0.039215,-0.193720,-0.368319,-1.053901,-0.110880,-0.334625,1.804794,0.207369,-0.213203,0.792435,-0.907890,0.206045,0.826788,-0.432596,-0.710615,-0.090628,-0.512033,-0.351646,-0.769104,-0.512588,-1.145987,-0.265536,0.360729,-0.631033,0.505256,-0.320193,-0.971318,0.465063,0.260622,-0.713288,0.195959,-0.559301,-0.636856,-0.824055,-0.252232,1.730523,-0.767444,...,-1.330281,-0.087328,-0.005858,-0.556388,-0.095049,0.542524,-0.540500,-0.471644,-0.002408,-0.190174,-1.200383,-0.266870,1.420096,0.189075,-0.529328,-0.609467,-0.566820,0.043313,1.205942,-0.235287,0.107570,-1.238209,2.251569,-0.196420,0.304925,-0.463835,-0.236676,-0.709219,-0.059789,0.312080,-0.361062,1.924569,0.301367,0.433249,-0.056081,-0.059811,-0.084596,-0.162037,-0.213925,-0.384486
3,0.665408,-0.136545,0.063771,0.348727,-0.584339,0.881179,-1.838868,-0.752402,-0.489295,-2.205337,-0.397183,0.088091,-1.993378,1.617620,-1.346301,0.407925,-1.049239,-0.689599,-0.858887,-0.761187,-0.173062,-2.078685,0.308963,-1.505522,2.397393,0.172499,-0.483800,0.107501,0.689927,-0.285608,0.793687,-1.621223,-0.233974,4.713715,-0.565749,-0.215680,-0.278301,-0.561325,1.070898,0.423454,...,0.002494,-0.669658,0.129397,-0.577301,-0.869072,-0.364

In [ ]:
encoded_train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1047 entries, 0 to 1046
Data columns (total 100 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rna_feature_0   1047 non-null   float32
 1   rna_feature_1   1047 non-null   float32
 2   rna_feature_2   1047 non-null   float32
 3   rna_feature_3   1047 non-null   float32
 4   rna_feature_4   1047 non-null   float32
 5   rna_feature_5   1047 non-null   float32
 6   rna_feature_6   1047 non-null   float32
 7   rna_feature_7   1047 non-null   float32
 8   rna_feature_8   1047 non-null   float32
 9   rna_feature_9   1047 non-null   float32
 10  rna_feature_10  1047 non-null   float32
 11  rna_feature_11  1047 non-null   float32
 12  rna_feature_12  1047 non-null   float32
 13  rna_feature_13  1047 non-null   float32
 14  rna_feature_14  1047 non-null   float32
 15  rna_feature_15  1047 non-null   float32
 16  rna_feature_16  1047 non-null   float32
 17  rna_feature_17  1047 non-null   

In [ ]:
encoded_train1['CELL_LINE_NAME'] = rna_y
encoded_train1

,rna_feature_0,rna_feature_1,rna_feature_2,rna_feature_3,rna_feature_4,rna_feature_5,rna_feature_6,rna_feature_7,rna_feature_8,rna_feature_9,rna_feature_10,rna_feature_11,rna_feature_12,rna_feature_13,rna_feature_14,rna_feature_15,rna_feature_16,rna_feature_17,rna_feature_18,rna_feature_19,rna_feature_20,rna_feature_21,rna_feature_22,rna_feature_23,rna_feature_24,rna_feature_25,rna_feature_26,rna_feature_27,rna_feature_28,rna_feature_29,rna_feature_30,rna_feature_31,rna_feature_32,rna_feature_33,rna_feature_34,rna_feature_35,rna_feature_36,rna_feature_37,rna_feature_38,rna_feature_39,...,rna_feature_62,rna_feature_63,rna_feature_64,rna_feature_65,rna_feature_66,rna_feature_67,rna_feature_68,rna_feature_69,rna_feature_70,rna_feature_71,rna_feature_72,rna_feature_73,rna_feature_74,rna_feature_75,rna_feature_76,rna_feature_77,rna_feature_78,rna_feature_79,rna_feature_80,rna_feature_81,rna_feature_82,rna_feature_83,rna_feature_84,rna_feature_85,rna_feature_86,rna_feature_87,rna_feature_88,rna_feature_89,rna_feature_90,rna_feature_91,rna_feature_92,rna_feature_93,rna_feature_94,rna_feature_95,rna_feature_96,rna_feature_97,rna_feature_98,rna_feature_99,Cell_Line_Name,CELL_LINE_NAME
0,-0.113710,0.015776,0.255900,-0.353810,-0.684555,-0.505022,-0.934096,-0.262980,0.328180,-0.888463,-0.213263,-0.112308,-0.193263,-0.697479,-0.204158,-0.475362,-0.459413,-0.713710,-0.043445,-0.604697,-0.441277,-0.326982,-0.331256,-0.468288,-0.481006,2.817312,3.122547,0.326295,-0.087718,-0.157410,-0.170199,-0.666725,-0.786257,-0.207826,-0.336046,-0.392179,-0.224414,0.245889,-0.173383,-0.487124,...,3.221149,-0.376878,-0.411979,-0.279865,-0.261245,-0.345919,-1.033504,-0.168441,-2.014476,-0.452390,1.168029,-0.148469,-1.168720,-0.884369,-0.106900,1.025782,-0.015760,-1.974355,-0.704214,-0.548893,-0.427587,-0.700450,-0.257901,-0.387663,-0.345593,-1.205233,-0.079529,-0.405452,0.008337,-0.692550,-0.049275,-0.354898,-0.261631,0.314960,-0.014273,-0.237338,-0.313053,-0.426225,NaN,NaN
1,-0.025810,-0.012047,-0.304307,0.070642,-0.540065,-0.388465,-1.128392,-0.104629,-0.889848,-0.851125,-0.287239,0.048427,-0.069018,-1.194497,-0.095489,-0.241955,-0.437581,-0.493676,0.198132,-0.078260,-0.344380,-1.193587,-0.721444,-0.492744,0.355221,-0.191624,-0.895223,0.531082,4.812373,-0.961698,-0.400196,-0.368726,-0.836760,-0.073466,-0.350438,0.095026,-0.513887,-0.888748,1.276347,-0.465436,...,-0.021343,-0.469225,-0.094063,0.020835,-0.649219,-0.573378,-1.085598,-0.440069,-1.363173,-0.543473,1.204380,-0.234619,-0.419783,-0.287695,0.392210,-0.205331,2.549847,0.056657,0.104438,-1.500150,-0.361864,0.433886,1.374583,-0.607971,-0.454727,-1.956478,-0.516387,0.583284,-0.262945,1.225905,0.551230,-0.066023,-0.161026,-0.626396,-0.135961,1.877104,-0.575734,-0.646045,NaN,NaN
2,-0.106869,0.272746,0.061858,-0.039215,-0.193720,-0.368319,-1.053901,-0.110880,-0.334625,1.804794,0.207369,-0.213203,0.792435,-0.907890,0.206045,0.826788,-0.432596,-0.710615,-0.090628,-0.512033,-0.351646,-0.769104,-0.512588,-1.145987,-0.265536,0.360729,-0.631033,0.505256,-0.320193,-0.971318,0.465063,0.260622,-0.713288,0.195959,-0.559301,-0.636856,-0.824055,-0.252232,1.730523,-0.767444,...,-0.005858,-0.556388,-0.095049,0.542524,-0.540500,-0.471644,-0.002408,-0.190174,-1.200383,-0.266870,1.420096,0.189075,-0.529328,-0.609467,-0.566820,0.043313,1.205942,-0.235287,0.107570,-1.238209,2.251569,-0.196420,0.304925,-0.463835,-0.236676,-0.709219,-0.059789,0.312080,-0.361062,1.924569,0.301367,0.433249,-0.056081,-0.059811,-0.084596,-0.162037,-0.213925,-0.384486,NaN,NaN
3,0.665408,-0.136545,0.063771,0.348727,-0.584339,0.881179,-1.838868,-0.752402,-0.489295,-2.205337,-0.397183,0.088091,-1.993378,1.617620,-1.346301,0.407925,-1.049239,-0.689599,-0.858887,-0.761187,-0.173062,-2.078685,0.308963,-1.505522,2.397393,0.172499,-0.483800,0.107501,0.689927,-0.285608,0.793687,-1.621223,-0.233974,4.713715,-0.565749,-0.215680,-0.278301,-0.561325,1.070898,0.423454,...,0.129397,-0.577301,-0.869072,-0.364203,-1.579021,-0.208154,-1.952604,0.050320,-0.593375,0.

In [ ]:
encoded_train1['CELL_LINE_NAME'] = rna_y
encoded_train1

In [ ]:
encoded_train1['CELL_LINE_NAME'] = rna_y

from google.colab import files
encoded_train1.to_csv('rna_encoded.csv') 
files.download('rna_encoded.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
rna_y

MEC-1                    MEC-1
NBsusSR                NBsusSR
M14                        M14
MDA-MB-134-VI    MDA-MB-134-VI
MCC26                    MCC26
                     ...      
H9                          H9
P3HR-1                  P3HR-1
HUH-6-clone5      HUH-6-clone5
GR-ST                    GR-ST
MM1S                      MM1S
Name: CELL_LINE_NAME, Length: 1047, dtype: object

In [ ]:
from google.colab import files
rna_y.to_csv('rna_y.csv') 
files.download('rna_y.csv')

In [ ]:
encoded_train1['Cell_Line_Name'] = rna_y
encoded_train1


In [ ]:
l11 = []
for x in drug_features.columns:
    
    if drug_features[x].dtype == object:
        l11.append(x)
print(l11)

In [ ]:
#drug_features.drop(index =203, inplace = True) # this row or drug was throwing continuous errors , so deleted from drug list

In [ ]:
drug_features["VR3_Dt"] = drug_features.VR3_Dt.astype(float)

In [ ]:
drug_features["gmin"] = drug_features.gmin.astype(float)
drug_features.drop(columns = ["Name"], inplace = True)
drug_features.head()

In [ ]:
drug_features["VR3_D"] = drug_features.VR3_D.astype(float)

In [ ]:

# checking for columns with same value across 223 drugs
same_value =[]
new_features=[]
for x in drug_features.columns:
    if drug_features[x].nunique() ==1:
        same_value.append(x)
    else :
        new_features.append(x)

In [ ]:
print(len(new_features))
print(len(same_value))
Drug_features_columns = drug_features.columns


In [ ]:
# checking for any empty columns
l1=[]
l2 =[]
for x in drug_features.columns:
        if len(drug_features[x].value_counts()) >0:
           l1.append(x)
        else:
           l2.append(x)
print(len(l1))
print(len(l2))
           

In [ ]:
new_Drug_features = drug_features[new_features]

In [ ]:

import numpy as np
new_Drug_features = new_Drug_features.replace([np.inf, -np.inf], np.nan)

In [ ]:
new_Drug_features.isnull().values.any()

In [ ]:
new_Drug_features.fillna(0, inplace =True)

In [ ]:
new_Drug_features.isnull().values.any()

In [ ]:
y = new_Drug_features['PubCHEM']
new_Drug_features.drop(columns =['PubCHEM'], inplace = True)
y

In [ ]:
new_Drug_features.shape

In [ ]:
train_scaled = minmax_scale(new_Drug_features, axis = 0)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(train_scaled, y, train_size = 0.9, random_state = seed(2017))

In [ ]:
### Define the encoder dimension
encoding_dim = 3

In [ ]:
X_train

In [ ]:
ncol = train_scaled.shape[1]

In [ ]:
input_dim = Input(shape = (ncol, ))

# Encoder Layers
#encoded1 = Dense(3000, activation = 'relu')(input_dim)
#encoded2 = Dense(2750, activation = 'relu')(encoded1)
#encoded3 = Dense(2500, activation = 'relu')(encoded2)
#encoded4 = Dense(2250, activation = 'relu')(encoded3)
#encoded5 = Dense(2000, activation = 'relu')(encoded4)
#encoded6 = Dense(1750, activation = 'relu')(encoded5)
encoded7 = Dense(1500, activation = 'relu')(input_dim)
encoded8 = Dense(1250, activation = 'relu')(encoded7)
encoded9 = Dense(1000, activation = 'relu')(encoded8)
encoded10 = Dense(750, activation = 'relu')(encoded9)
encoded11 = Dense(500, activation = 'relu')(encoded10)
encoded12 = Dense(250, activation = 'relu')(encoded11)
encoded13 = Dense(encoding_dim, activation = 'relu')(encoded12)

# Decoder Layers
decoded1 = Dense(250, activation = 'relu')(encoded13)
decoded2 = Dense(500, activation = 'relu')(decoded1)
decoded3 = Dense(750, activation = 'relu')(decoded2)
decoded4 = Dense(1000, activation = 'relu')(decoded3)
decoded5 = Dense(1250, activation = 'relu')(decoded4)
decoded6 = Dense(1500, activation = 'relu')(decoded5)
#decoded7 = Dense(1750, activation = 'relu')(decoded6)
#decoded8 = Dense(2000, activation = 'relu')(decoded7)
#decoded9 = Dense(2250, activation = 'relu')(decoded8)
#decoded10 = Dense(2500, activation = 'relu')(decoded9)
#decoded11 = Dense(2750, activation = 'relu')(decoded10)
#decoded12 = Dense(3000, activation = 'relu')(decoded11)
decoded13 = Dense(ncol, activation = 'relu')(decoded6)

# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded13)

# Compile the Model
autoencoder.compile(optimizer = 'adam', loss = 'binary_crossentropy')

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.fit(X_train, X_train, epochs = 50, batch_size = 32, shuffle = False, validation_data = (X_test, X_test))

In [ ]:
encoder = Model(inputs = input_dim, outputs = encoded13)
encoded_input = Input(shape = (encoding_dim, ))

In [ ]:
encoded_train = pd.DataFrame(encoder.predict(train_scaled))
encoded_train = encoded_train.add_prefix('feature_')

In [ ]:
encoded_train

In [ ]:
encoded_train_columns = encoded_train.columns
for l in encoded_train_columns:
  if (encoded_train[l] == 0).all():
    print("TRUE", l)